# Preparation

In [1]:
!pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk &>/dev/null
!pip install git+https://github.com/mimoralea/gym-aima#egg=gym-aima &>/dev/null

In [2]:
import gym
import gym_walk, gym_aima
import numpy as np
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

from itertools import cycle, count
import itertools
from tabulate import tabulate

import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
SEEDS = (12, 34, 56, 78, 90)

%matplotlib inline
import warnings

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
params = {
    'figure.figsize': (15, 8),
    'font.size': 24,
    'legend.fontsize': 20,
    'axes.titlesize': 28,
    'axes.labelsize': 24,
    'xtick.labelsize': 20,
    'ytick.labelsize': 20
}
pylab.rcParams.update(params)
np.set_printoptions(suppress=True)

# Utilities

In [3]:
# slightly modified from chapter 4 - pi is lambda instead of dict
def value_iteration(P, gamma=1.0, theta=1e-10):
  n_states = len(P)
  n_actions = len(P[0])
  # initialise V to zeros
  V = np.zeros(n_states, dtype=np.float64)

  while True:
    # initialise Q-function to zeros
    Q = np.zeros((n_states, n_actions), dtype=np.float64)
    for s in range(n_states):
      for a in range(n_actions):
        for prob, next_state, reward, done in P[s][a]:
          Q[s,a] += prob * (reward + gamma * V[next_state] * (not done))
    if np.max( np.abs(V - np.max(Q, axis=1)) ) < theta:
      break; # if the action-advantage function converged, break

    V = np.max(Q, axis=1) # combination of policy eval and improv
  # pi = {s:a for s, a in enumerate( np.argmax(Q, axis=1) )}
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

  return Q, V, pi

def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
  print(title)
  arrs = {k:v for k,v in enumerate(action_symbols)}
  for s in range(len(P)):
    a = pi(s)
    print("| ", end="")
    if np.all([done for action in P[s].values() for _, _, _, done in action]):
        print("".rjust(9), end=" ")
    else:
        print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
    if (s + 1) % n_cols == 0: print("|")

def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
  print(title)
  for s in range(len(P)):
    v = V[s]
    print("| ", end="")
    if np.all([done for action in P[s].values() for _, _, _, done in action]):
        print("".rjust(9), end=" ")
    else:
        print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
    if (s + 1) % n_cols == 0: print("|")

def print_action_value_function(Q,
                                optimal_Q=None,
                                action_symbols=('<', '>'),
                                prec=3,
                                title='Action-value function:'):
  vf_types=('',) if optimal_Q is None else ('', '*', 'er')
  headers = ['s',] + [' '.join(i) for i in list(itertools.product(vf_types, action_symbols))]
  print(title)
  states = np.arange(len(Q))[..., np.newaxis]
  arr = np.hstack((states, np.round(Q, prec)))
  if not (optimal_Q is None):
    arr = np.hstack((arr, np.round(optimal_Q, prec), np.round(optimal_Q-Q, prec)))
  print(tabulate(arr, headers, tablefmt="fancy_grid"))

"""
run the policy pi in the env for a number of episodes, collect and return
results: % reached goal, avg reward, avg regret
"""
def get_policy_metrics(env, gamma, pi, goal_state, optimal_Q,
                       n_episodes=100, max_steps=200):
  random.seed(123); np.random.seed(123) ; env.seed(123)
  reached_goal, episode_reward, episode_regret = [], [], []
  for _ in range(n_episodes):
    state, done, steps = env.reset(), False, 0
    episode_reward.append(0.0)
    episode_regret.append(0.0)
    while not done and steps < max_steps:
      action = pi(state)
      regret = np.max(optimal_Q[state]) - optimal_Q[state][action]
      episode_regret[-1] += regret

      state, reward, done, _ = env.step(action)
      episode_reward[-1] += (gamma**steps * reward)

      steps += 1

    reached_goal.append(state == goal_state)
  results = np.array((np.sum(reached_goal)/len(reached_goal)*100,
                      np.mean(episode_reward),
                      np.mean(episode_regret)))
  return results

def get_metrics_from_tracks(env, gamma, goal_state, optimal_Q, pi_track, coverage=0.1):
  total_samples = len(pi_track)
  n_samples = int(total_samples * coverage)
  samples_e = np.linspace(0, total_samples, n_samples, endpoint=True, dtype=np.int)
  metrics = []
  for e, pi in enumerate(tqdm(pi_track)):
    if e in samples_e:
      metrics.append(get_policy_metrics(
        env,
        gamma=gamma,
        pi=lambda s: pi[s],
        goal_state=goal_state,
        optimal_Q=optimal_Q))
    else:
      metrics.append(metrics[-1])
  metrics = np.array(metrics)
  success_rate_ma, mean_return_ma, mean_regret_ma = np.apply_along_axis(moving_average, axis=0, arr=metrics).T
  return success_rate_ma, mean_return_ma, mean_regret_ma

def mean_return(env, gamma, pi, n_episodes=100, max_steps=200):
  random.seed(123); np.random.seed(123) ; env.seed(123)
  results = []
  for _ in range(n_episodes):
    state, done, steps = env.reset(), False, 0
    results.append(0.0)
    while not done and steps < max_steps:
      state, reward, done, _ = env.step(pi(state))
      results[-1] += (gamma**steps * reward)
      steps += 1
  return np.mean(results)

def rmse(x, y, dp=4):
    return np.round(np.sqrt(np.mean((x - y)**2)), dp)

"""exponentially decaying schedule
this function allows you to calculate all the values for alpha for the full training process
"""
def decay_schedule(init_value, min_value,
                   decay_ratio, # determines how many episodes to use for decay
                   max_steps, # i.e. n_episodes in previous chapters
                   log_start=-2, log_base=10):
  assert min_value<=init_value, "min_value must be <= init_value"
  decay_steps = max(int(max_steps*decay_ratio), 1)
  rem_steps = max_steps - decay_steps # remaining steps (i.e. not used for decay)

  # calculate actual values of an inverse log curve ([::-1] reverse the order)
  values = np.logspace(start=log_start, stop=0,
                       num=decay_steps, # number of samples to generate
                       base=log_base,
                       endpoint=True # samples are inclusive of 'stop'
                       )[::-1]
  # print("reverse logspace: ", values)
  # normalise to between 0 and 1
  values = (values - values.min()) / (values.max() - values.min())
  # transform the points to lay between init_value and min_value
  values = min_value + (init_value - min_value) * values
  values = np.pad(values, (0, rem_steps), 'edge')
  return values

## Plotting functions

In [4]:
def plot_value_function(title, V_track, V_true=None, log=False, limit_value=0.05, limit_items=5):
  np.random.seed(123)
  per_col = 25
  linecycler = cycle(["-","--",":","-."])
  legends = []

  valid_values = np.argwhere(V_track[-1] > limit_value).squeeze()
  items_idxs = np.random.choice(valid_values,
                                min(len(valid_values), limit_items),
                                replace=False)
  # draw the true values first
  if V_true is not None:
    for i, state in enumerate(V_track.T):
      if i not in items_idxs:
        continue
      if state[-1] < limit_value:
        continue

      label = 'v*({})'.format(i)
      plt.axhline(y=V_true[i], color='k', linestyle='-', linewidth=1)
      plt.text(int(len(V_track)*1.02), V_true[i]+.01, label)

  # then the estimates
  for i, state in enumerate(V_track.T):
    if i not in items_idxs:
      continue
    if state[-1] < limit_value:
      continue
    line_type = next(linecycler)
    label = 'V({})'.format(i)
    p, = plt.plot(state, line_type, label=label, linewidth=3)
    legends.append(p)

  legends.reverse()

  ls = []
  for loc, idx in enumerate(range(0, len(legends), per_col)):
    subset = legends[idx:idx+per_col]
    l = plt.legend(subset, [p.get_label() for p in subset],
                  loc='center right', bbox_to_anchor=(1.25, 0.5))
    ls.append(l)
  [plt.gca().add_artist(l) for l in ls[:-1]]
  if log: plt.xscale('log')
  plt.title(title)
  plt.ylabel('State-value function')
  plt.xlabel('Episodes (log scale)' if log else 'Episodes')
  plt.show()

def plot_transition_model(T_track, episode = 0):
  fig = plt.figure(figsize=(20,10))
  ax = fig.add_subplot(111, projection='3d')
  ax.view_init(elev=20, azim=50)

  color_left = '#008fd5' # ax._get_lines.get_next_color()
  color_right = '#fc4f30' #ax._get_lines.get_next_color()

  left_prob = np.divide(T_track[episode][:,0].T,
                        T_track[episode][:,0].sum(axis=1).T).T
  left_prob = np.nan_to_num(left_prob, 0)

  right_prob = np.divide(T_track[episode][:,1].T,
                          T_track[episode][:,1].sum(axis=1).T).T
  right_prob = np.nan_to_num(right_prob, 0)

  for s in np.arange(9):
    ax.bar3d(s+0.1, np.arange(9)+0.1, np.zeros(9),
            np.zeros(9)+0.3,
            np.zeros(9)+0.3,
            left_prob[s],
            color=color_left,
            alpha=0.75,
            shade=True)
    ax.bar3d(s+0.1, np.arange(9)+0.1, left_prob[s],
            np.zeros(9)+0.3,
            np.zeros(9)+0.3,
            right_prob[s],
            color=color_right,
            alpha=0.75,
            shade=True)

  ax.tick_params(axis='x', which='major', pad=10)
  ax.tick_params(axis='y', which='major', pad=10)
  ax.tick_params(axis='z', which='major', pad=10)

  ax.xaxis.set_rotate_label(False)
  ax.yaxis.set_rotate_label(False)
  ax.zaxis.set_rotate_label(False)
  ax.set_xticks(np.arange(9))
  ax.set_yticks(np.arange(9))

  plt.title('SWS learned MDP after {} episodes'.format(episode+1))
  ax.set_xlabel('Initial\nstate', labelpad=75, rotation=0)
  ax.set_ylabel('Landing\nstate', labelpad=75, rotation=0)
  ax.set_zlabel('Transition\nprobabilities', labelpad=75, rotation=0)

  left_proxy = plt.Rectangle((0, 0), 1, 1, fc=color_left)
  right_proxy = plt.Rectangle((0, 0), 1, 1, fc=color_right)

  plt.legend((left_proxy, right_proxy),
            ('Left', 'Right'),
            bbox_to_anchor=(0.15, 0.9),
            borderaxespad=0.)

  ax.dist = 12
  #plt.gcf().subplots_adjust(left=0.1, right=0.9)
  plt.tight_layout()

  plt.show()

def plot_model_state_sampling(planning, algo='Dyna-Q'):
  fig = plt.figure(figsize=(20,10))

  color_left = '#008fd5' # ax._get_lines.get_next_color()
  color_right = '#fc4f30' #ax._get_lines.get_next_color()

  for s in np.arange(9):
    actions = planning[np.where(planning[:,0]==s)[0], 1]
    left = len(actions[actions == 0])
    right = len(actions[actions == 1])
    plt.bar(s, right, 0.2, color=color_right)
    plt.bar(s, left, 0.2, color=color_left, bottom=right)


  plt.title('States samples from {}\nlearned model of SWS environment'.format(algo))
  plt.xticks(range(9))
  plt.xlabel('Initial states sampled', labelpad=20)
  plt.ylabel('Count', labelpad=50, rotation=0)

  left_proxy = plt.Rectangle((0, 0), 1, 1, fc=color_left)
  right_proxy = plt.Rectangle((0, 0), 1, 1, fc=color_right)

  plt.legend((left_proxy, right_proxy),
            ('Left', 'Right'),
            bbox_to_anchor=(0.99, 1.1),
            borderaxespad=0.)

  #plt.gcf().subplots_adjust(left=0.1, right=0.9)
  plt.tight_layout()

  plt.show()

def plot_model_state_7(planning, algo='Dyna-Q'):
  fig = plt.figure(figsize=(20,10))

  color_left = '#008fd5' # ax._get_lines.get_next_color()
  color_right = '#fc4f30' #ax._get_lines.get_next_color()


  state_7 = planning[np.where(planning[:,0]==7)]
  for sp in [6, 7, 8]:
    actions = state_7[np.where(state_7[:,3]==sp)[0], 1]
    left = len(actions[actions == 0])
    right = len(actions[actions == 1])
    plt.bar(sp, right, 0.2, color=color_right)
    plt.bar(sp, left, 0.2, color=color_left, bottom=right)

  plt.title('Next states samples by {}\nin SWS environment from state 7'.format(algo))
  plt.xticks([6,7,8])
  plt.xlabel('Landing states', labelpad=20)
  plt.ylabel('Count', labelpad=50, rotation=0)

  left_proxy = plt.Rectangle((0, 0), 1, 1, fc=color_left)
  right_proxy = plt.Rectangle((0, 0), 1, 1, fc=color_right)

  plt.legend((left_proxy, right_proxy),
            ('Left', 'Right'),
            bbox_to_anchor=(0.99, 1.1),
            borderaxespad=0.)

  #plt.gcf().subplots_adjust(left=0.1, right=0.9)
  plt.tight_layout()

  plt.show()

In the replace-trace strategy, traces are set to 1 when a state-action pair is visited, and decay based on \lambda value just like accumulate-trace strategy (in chapter 5).

Diff between replace-trace and accumulate-trace:
* accumulate-trace tracks the visited states; the eligibility trace is without bound
* replace-trace tracks the visited state-action pair; also the eligibility trace is clipped to 1 to avoid dead loop

# Slippery Walk Seven

In [9]:
env = gym.make('SlipperyWalkSeven-v0')
init_state = env.reset()
goal_state = 8
gamma = 0.99
n_episodes = 3000
P = env.env.P
n_cols, svf_prec, err_prec, avf_prec=9, 4, 2, 3
action_symbols=('<', '>')
limit_items, limit_value = 5, 0.0
cu_limit_items, cu_limit_value, cu_episodes = 10, 0.0, 100

optimal_Q, optimal_V, optimal_pi = value_iteration(P, gamma=gamma)
print_state_value_function(optimal_V, P, n_cols=n_cols, prec=svf_prec, title='Optimal state-value function:')
print()

print_action_value_function(optimal_Q,
                            None,
                            action_symbols=action_symbols,
                            prec=avf_prec,
                            title='Optimal action-value function:')
print()
print_policy(optimal_pi, P, action_symbols=action_symbols, n_cols=n_cols)
success_rate_op, mean_return_op, mean_regret_op = get_policy_metrics(
    env, gamma=gamma, pi=optimal_pi, goal_state=goal_state, optimal_Q=optimal_Q)
print('Reaches goal {:.2f}%. Obtains an average return of {:.4f}. Regret of {:.4f}'.format(
    success_rate_op, mean_return_op, mean_regret_op))

Optimal state-value function:
|           | 01 0.5637 | 02  0.763 | 03 0.8449 | 04 0.8892 | 05  0.922 | 06 0.9515 | 07 0.9806 |           |

Optimal action-value function:
╒═════╤═══════╤═══════╕
│   s │     < │     > │
╞═════╪═══════╪═══════╡
│   0 │ 0     │ 0     │
├─────┼───────┼───────┤
│   1 │ 0.312 │ 0.564 │
├─────┼───────┼───────┤
│   2 │ 0.67  │ 0.763 │
├─────┼───────┼───────┤
│   3 │ 0.803 │ 0.845 │
├─────┼───────┼───────┤
│   4 │ 0.864 │ 0.889 │
├─────┼───────┼───────┤
│   5 │ 0.901 │ 0.922 │
├─────┼───────┼───────┤
│   6 │ 0.932 │ 0.952 │
├─────┼───────┼───────┤
│   7 │ 0.961 │ 0.981 │
├─────┼───────┼───────┤
│   8 │ 0     │ 0     │
╘═════╧═══════╧═══════╛

Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > | 06      > | 07      > |           |
Reaches goal 96.00%. Obtains an average return of 0.8672. Regret of 0.0000


# Sarsa(λ)

This is a mix of Sarsa and TD(λ) methods.

In [5]:
def sarsa_lambda(env, gamma=1.0,
                 init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
                 init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
                 lambda_=0.5, # lambda is reserved word in python, so add _
                 replacing_traces=True,
                 n_episodes=3000):
  nS = env.observation_sapece.n
  nA = env.action_space.n
  pi_track = []
  Q = np.zeros((nS, nA), dtype=np.float32)
  Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float32)
  # eligibility trace keeps track of state-action pairs
  E = np.zeros((nS, nA), dtype=np.float32)
  select_action = lambda state, Q, epsilon: \
    np.argmax(Q[state]) if np.random.random() > epsilon \
    else np.random.randint(len(Q[state])) # random action of the state

  alphas = decay_schedule(init_alpha, min_alpha, alphs_decay_ratio, n_episodes)
  epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

  for e in tqdm(range(n_episodes), leave=False):
    E.fill(0) # at every episode, we reset eligitability of every state-action to 0
    state = env.reset()
    done = False
    action = select_action(state, Q, epsilons[e])

    while not done:
      # collect experience
      next_state, reward, done, _ = env.step(action)
      next_action = select_action(next_state, Q, epsilons[e])

      # same as original sarsa
      td_target = reward + gamma * Q[next_state][next_action] * (not done)
      td_error = td_target - Q[state][action]

      # increment state-action pair trace, clip to 1 if it's replacing trace
      E[state][action]+=1
      if replacing_traces: E.clip(0,1, out=E)

      # notice we update entire Q table for all eligible state-action pairs
      Q = Q + alphas[e] * td_error * E
      E = gamma * lambda_ * E # decay E

      state, action = next_state, next_action

    Q_track[e] = Q
    pi_track.append(np.argmax(Q[state], axis=1)) # axis 1 is action

  V = np.max(Q, axis=1)
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

  return Q, V, pi, Q_track, pi_track

# Watkin's Q(λ)

it's an off-policy control version of λ algorithms. Q(λ) is Q-learning using λ return for policy evaluation of the GPI pattern.

In [6]:
def q_lambda(env, gamma=1.0,
             init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
             init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
             lambda_=0.5,
             replacing_traces=True,
             n_episodes=3000):
  nS = env.observation_space.n
  nA = env.action_space.n
  pi_track=[]
  Q = np.zeros((nS, nA), dtype=np.float32)
  Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float32)
  E = np.zeros((nS, nA), dtype=np.float32)
  select_action = lambda state, Q, epsilon: \
    np.argmax(Q[state]) if np.random.random() > epsilon \
    else np.random.randint(Q[state])
  alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
  epsilons = decay_schedule(init_epsilon, min_epsilon, epsilonn_decay_ratio, n_episodes)

  for e in tqdm(range(n_episodes), leave=False):
    E.fill(0)
    state = env.reset()
    done = False
    action = select_action(state, Q, epsilons[e])
    while not done:
      next_state, reward, done, _ = env.step(action)
      next_action = select_action(next_state, Q, epsilons[e])
      # verify that action on next step is still from greedy policy
      next_action_is_greedy = Q[next_state][next_action] == Q[next_state].max()
      td_target = reward + gamma * Q[next_state].max() * (not done)
      td_error = td_target - Q[state][action] # note: use current state

      if replacing_traces: E[state].fill(0)

      E[state][action] += 1
      Q = Q + alphas[e] * td_error + E

      if next_action_is_greedy:
        E *= gamma * lambda_ # decay E as usual
      else:
        E.fill(0) # reset E because we want to learn greedy policy

      state, action = next_state, next_action

    Q_track[e] = Q
    pi_track.append(np.max(Q, axis=1))

  V = np.max(Q, axis=1)
  pi = lambda s: {s:a for s,a in enumerate(np.argmax(Q, axis=1))}[s]
  return Q

# Dyna-Q
Unifying model-free and model-based/planning methods  by interleaving a model-free method (Q-learning) and a planning method (similar to Value Iteration).

In [7]:
def dyna_q(env, gamma=1.0,
           init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
           init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
           n_planning=3, # number of updates to the estimates to run from learned model
           n_episodes=3000
           ):
  nS = env.observation_space.n
  nA = env.action_space.n
  Q = np.zeros((nS, nA), dtype=np.float32)
  Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float32)

  # keep track of the transition function
  T_count = np.zeros((nS, nA, nS), dtype=np.int16)
  # keep track of reward signal
  R_model = np.zeros((nS, nA, nS), dtype=np.float32)

  select_action = lambda state, Q, epsilon: \
    np.argmax(Q[state]) if np.random.random()>epsilon \
    else np.random().randint(len(Q[state]))

  alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
  epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

  for e in tqdm(range(n_episodes), leave=False):
    state = env.reset()
    done = False
    while not done:
      action = select_action(state, Q, epsilon[e])
      next_state, reward, done, _ = env.step(action)

      # count number of full transitions, the (s, a, s') tuple
      T_count[state][action][next_state] += 1

      r_diff = reward - R_model[state][action][next_state]
      R_model[state][action][next_state] += r_diff / T_count[state][action][next_state]

      # same as Q-learning (off-policy, using the max)
      td_target = reward + gamma * Q[next_state].max() * (not done)
      td_error = td_target - Q[state][action]
      Q[state][action] += alphas[e] * td_error
